# Data anonymization with Microsoft Presidio | Microsoft Presidioによるデータ匿名化

[![Colabで開く](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/guides/privacy/presidio_data_anonymization/index.ipynb)

> [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/guides/privacy/presidio_data_anonymization/index.ipynb)

> [Presidio](https://microsoft.github.io/presidio/)（ラテン語のpraesidium「保護、駐屯地」からの由来）は、機密データが適切に管理・統制されることを支援します。クレジットカード番号、名前、場所、社会保障番号、ビットコインウォレット、アメリカの電話番号、財務データなど、テキストや画像に含まれるプライベートな情報の迅速な識別と匿名化モジュールを提供します。
>
> > [Presidio](https://microsoft.github.io/presidio/) (Origin from Latin praesidium ‘protection, garrison’) helps to ensure sensitive data is properly managed and governed. It provides fast identification and anonymization modules for private entities in text and images such as credit card numbers, names, locations, social security numbers, bitcoin wallets, US phone numbers, financial data and more.

## Use case | ユースケース

GPT-4のような言語モデルに情報を渡す前にデータの匿名化は重要です。なぜなら、プライバシーを保護し、機密性を維持するのに役立つからです。データが匿名化されていない場合、名前、住所、連絡先番号、その他の特定個人にリンクされた識別子などの機密情報が学習され、悪用される可能性があります。したがって、この個人を特定できる情報（PII）を曖昧にするか削除することで、個人のプライバシー権を侵害することなく、またデータ保護法規を違反することなく、データを自由に使用できます。

> Data anonymization is crucial before passing information to a language model like GPT-4 because it helps protect privacy and maintain confidentiality. If data is not anonymized, sensitive information such as names, addresses, contact numbers, or other identifiers linked to specific individuals could potentially be learned and misused. Hence, by obscuring or removing this personally identifiable information (PII), data can be used freely without compromising individuals' privacy rights or breaching data protection laws and regulations.

## Overview | 概要

匿名化は二つのステップから成り立っています：

> Anonynization consists of two steps:

1. **識別:** 個人を特定できる情報（PII）を含むすべてのデータフィールドを特定してください。

   > **Identification:** Identify all data fields that contain personally identifiable information (PII).

2. **置換**: 個人情報を示すPIIを、個人に関する情報を漏らさない擬似的な値やコードに置き換えます。これにより、参照用として使用することが可能です。通常の暗号化は使用していません。なぜなら、言語モデルは暗号化されたデータの意味や文脈を理解することができないためです。

   > **Replacement**: Replace all PIIs with pseudo values or codes that do not reveal any personal information about the individual but can be used for reference. We're not using regular encryption, because the language model won't be able to understand the meaning or context of the encrypted data.


匿名化の目的で、*Microsoft Presidio*と*Faker*フレームワークを組み合わせて使用しています。これらが提供する機能の幅広さが理由です。完全な実装は`PresidioAnonymizer`で利用可能です。

> We use *Microsoft Presidio* together with *Faker* framework for anonymization purposes because of the wide range of functionalities they provide. The full implementation is available in `PresidioAnonymizer`.

## Quickstart | クイックスタート

以下に、LangChainで匿名化を活用するユースケースを示します。

> Below you will find the use case on how to leverage anonymization in LangChain.




In [1]:
# Install necessary packages
# ! pip install langchain langchain-experimental openai presidio-analyzer presidio-anonymizer spacy Faker
# ! python -m spacy download en_core_web_lg

PII匿名化がどのように機能するか、サンプル文を使って見てみましょう：

> \
> Let's see how PII anonymization works using a sample sentence:




In [2]:
from langchain_experimental.data_anonymizer import PresidioAnonymizer

anonymizer = PresidioAnonymizer()

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is James Martinez, call me at (576)928-1972x679 or email me at lisa44@example.com'

### Using with LangChain Expression Language | LangChain Expression Languageの使用方法

LCELを使用すると、匿名化をアプリケーションの他の部分と簡単に連携させることができます。

> With LCEL we can easily chain together anonymization with the rest of our application.




In [3]:
# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv

# dotenv.load_dotenv()

In [4]:
text = """Slim Shady recently lost his wallet. 
Inside is some cash and his credit card with the number 4916 0387 9536 0861. 
If you would find it, please call at 313-666-7440 or write an email here: real.slim.shady@gmail.com."""

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

anonymizer = PresidioAnonymizer()

template = """Rewrite this text into an official, short email:

{anonymized_text}"""
prompt = PromptTemplate.from_template(template)
llm = ChatOpenAI(temperature=0)

chain = {"anonymized_text": anonymizer.anonymize} | prompt | llm
response = chain.invoke(text)
print(response.content)

Dear Sir/Madam,

We regret to inform you that Mr. Dennis Cooper has recently misplaced his wallet. The wallet contains a sum of cash and his credit card, bearing the number 3588895295514977. 

Should you happen to come across the aforementioned wallet, kindly contact us immediately at (428)451-3494x4110 or send an email to perryluke@example.com.

Your prompt assistance in this matter would be greatly appreciated.

Yours faithfully,

[Your Name]


## Customization | カスタマイズ

`analyzed_fields`を指定することで、特定のタイプのデータのみを匿名化することができます。

> We can specify `analyzed_fields` to only anonymize particular types of data.




In [6]:
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON"])

anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Shannon Steele, call me at 313-666-7440 or email me at real.slim.shady@gmail.com'

見ての通り、名前は正しく識別され、別のものに置き換えられました。`analyzed_fields` 属性は、どの値を検出し、そして置き換えるかを決定します。リストに *PHONE\_NUMBER* を追加することができます：

> As can be observed, the name was correctly identified and replaced with another. The `analyzed_fields` attribute is responsible for what values are to be detected and substituted. We can add *PHONE\_NUMBER* to the list:




In [7]:
anonymizer = PresidioAnonymizer(analyzed_fields=["PERSON", "PHONE_NUMBER"])
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Wesley Flores, call me at (498)576-9526 or email me at real.slim.shady@gmail.com'

analyzed\_fieldsが指定されていない場合、デフォルトでは匿名化ツールはすべてのサポートされるフォーマットを検出します。以下がそれらの完全なリストです：

> \
> If no analyzed\_fields are specified, by default the anonymizer will detect all supported formats. Below is the full list of them:

\['PERSON', 'EMAIL\_ADDRESS', 'PHONE\_NUMBER', 'IBAN\_CODE', 'CREDIT\_CARD', 'CRYPTO', 'IP\_ADDRESS', 'LOCATION', 'DATE\_TIME', 'NRP', 'MEDICAL\_LICENSE', 'URL', 'US\_BANK\_NUMBER', 'US\_DRIVER\_LICENSE', 'US\_ITIN', 'US\_PASSPORT', 'US\_SSN']

> `['PERSON', 'EMAIL_ADDRESS', 'PHONE_NUMBER', 'IBAN_CODE', 'CREDIT_CARD', 'CRYPTO', 'IP_ADDRESS', 'LOCATION', 'DATE_TIME', 'NRP', 'MEDICAL_LICENSE', 'URL', 'US_BANK_NUMBER', 'US_DRIVER_LICENSE', 'US_ITIN', 'US_PASSPORT', 'US_SSN']`

**免責事項：** 検出されるべきプライベートデータを慎重に定義することをお勧めします。Presidioは完璧に機能するわけではなく、時には誤りを犯すことがあるため、データに対するコントロールをより強化することが望ましいです。

> **Disclaimer:** We suggest carefully defining the private data to be detected - Presidio doesn't work perfectly and it sometimes makes mistakes, so it's better to have more control over the data.




In [8]:
anonymizer = PresidioAnonymizer()
anonymizer.anonymize(
    "My name is Slim Shady, call me at 313-666-7440 or email me at real.slim.shady@gmail.com"
)

'My name is Carla Fisher, call me at 001-683-324-0721x0644 or email me at krausejeremy@example.com'

上記の検出されたフィールドのリストが十分でない場合があります。例えば、既に利用可能な*PHONE\_NUMBER*フィールドはポーランドの電話番号をサポートしておらず、それを他のフィールドと混同してしまうことがあります。

> \
> It may be that the above list of detected fields is not sufficient. For example, the already available *PHONE\_NUMBER* field does not support polish phone numbers and confuses it with another field:




In [9]:
anonymizer = PresidioAnonymizer()
anonymizer.anonymize("My polish phone number is 666555444")

'My polish phone number is QESQ21234635370499'

その後、あなた自身の認識器を作成し、既存のプールに追加することができます。認識器を具体的にどのように作成するかは、[Presidio documentation](https://microsoft.github.io/presidio/samples/python/customizing_presidio_analyzer/)に記載されています。

> \
> You can then write your own recognizers and add them to the pool of those present. How exactly to create recognizers is described in the [Presidio documentation](https://microsoft.github.io/presidio/samples/python/customizing_presidio_analyzer/).




In [10]:
# Define the regex pattern in a Presidio `Pattern` object:
from presidio_analyzer import Pattern, PatternRecognizer

polish_phone_numbers_pattern = Pattern(
    name="polish_phone_numbers_pattern",
    regex="(?<!\w)(\(?(\+|00)?48\)?)?[ -]?\d{3}[ -]?\d{3}[ -]?\d{3}(?!\w)",
    score=1,
)

# Define the recognizer with one or more patterns
polish_phone_numbers_recognizer = PatternRecognizer(
    supported_entity="POLISH_PHONE_NUMBER", patterns=[polish_phone_numbers_pattern]
)

これで、匿名化ツールに `add_recognizer` メソッドを呼び出して、認識器を追加することができます。

> \
> Now, we can add recognizer by calling `add_recognizer` method on the anonymizer:




In [11]:
anonymizer.add_recognizer(polish_phone_numbers_recognizer)

さあ、パターンベースの認識機能を追加したことで、アノニマイザーはポーランドの電話番号も処理できるようになりました。

> \
> And voilà! With the added pattern-based recognizer, the anonymizer now handles polish phone numbers.




In [12]:
print(anonymizer.anonymize("My polish phone number is 666555444"))
print(anonymizer.anonymize("My polish phone number is 666 555 444"))
print(anonymizer.anonymize("My polish phone number is +48 666 555 444"))

My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>
My polish phone number is <POLISH_PHONE_NUMBER>


問題は、ポーランドの電話番号を認識できるようになったとしても、指定されたフィールドをどのように置き換えるかを指示するメソッド（オペレーター）がないため、出力では `<POLISH_PHONE_NUMBER>` という文字列のみを提供していることです。これを正しく置き換えるメソッドを作成する必要があります。

> \
> The problem is - even though we recognize polish phone numbers now, we don't have a method (operator) that would tell how to substitute a given field - because of this, in the outpit we only provide string `<POLISH_PHONE_NUMBER>` We need to create a method to replace it correctly:




In [13]:
from faker import Faker

fake = Faker(locale="pl_PL")


def fake_polish_phone_number(_=None):
    return fake.phone_number()


fake_polish_phone_number()

'665 631 080'

Fakerを使用して擬似データを作成しました。これでオペレーターを作成し、匿名化ツールに追加することができます。オペレーターの作成とその詳細情報については、[シンプル](https://microsoft.github.io/presidio/tutorial/10_simple_anonymization/)および[カスタム](https://microsoft.github.io/presidio/tutorial/11_custom_anonymization/)匿名化に関するPresidioのドキュメントをご覧ください。

> \
> We used Faker to create pseudo data. Now we can create an operator and add it to the anonymizer. For complete information about operators and their creation, see the Presidio documentation for [simple](https://microsoft.github.io/presidio/tutorial/10_simple_anonymization/) and [custom](https://microsoft.github.io/presidio/tutorial/11_custom_anonymization/) anonymization.




In [14]:
from presidio_anonymizer.entities import OperatorConfig

new_operators = {
    "POLISH_PHONE_NUMBER": OperatorConfig(
        "custom", {"lambda": fake_polish_phone_number}
    )
}

In [15]:
anonymizer.add_operators(new_operators)

In [16]:
anonymizer.anonymize("My polish phone number is 666555444")

'My polish phone number is 538 521 657'

## Important considerations | 重要な考慮事項

### Anonymizer detection rates | 匿名化検出率

匿名化のレベルと検出の精度は、実装された認識器の品質に応じて同じくらい良くなります。

> **The level of anonymization and the precision of detection are just as good as the quality of the recognizers implemented.**

異なるソースや言語からのテキストには様々な特徴がありますので、検出精度をテストし、より良い結果を得るために認識器やオペレーターを反復的に追加する必要があります。

> Texts from different sources and in different languages have varying characteristics, so it is necessary to test the detection precision and iteratively add recognizers and operators to achieve better and better results.

Microsoft Presidioは匿名化を洗練させるための多くの自由を提供しています。このライブラリの作者は、[検出率を向上させるための推奨事項と段階的なガイド](https://github.com/microsoft/presidio/discussions/767#discussion-3567223)を提供しています。

> Microsoft Presidio gives a lot of freedom to refine anonymization. The library's author has provided his [recommendations and a step-by-step guide for improving detection rates](https://github.com/microsoft/presidio/discussions/767#discussion-3567223).




### Instance anonymization | インスタンスの匿名化

`PresidioAnonymizer`には組み込みのメモリがありません。そのため、後続のテキストでエンティティが2回出現した場合、それぞれ異なる偽の値で置き換えられます：

> `PresidioAnonymizer` has no built-in memory. Therefore, two occurrences of the entity in the subsequent texts will be replaced with two different fake values:




In [17]:
print(anonymizer.anonymize("My name is John Doe. Hi John Doe!"))
print(anonymizer.anonymize("My name is John Doe. Hi John Doe!"))

My name is Robert Morales. Hi Robert Morales!
My name is Kelly Mccoy. Hi Kelly Mccoy!


以前の匿名化結果を保持するためには、組み込みメモリを持つ`PresidioReversibleAnonymizer`を使用してください。

> To preserve previous anonymization results, use `PresidioReversibleAnonymizer`, which has built-in memory:




In [18]:
from langchain_experimental.data_anonymizer import PresidioReversibleAnonymizer

anonymizer_with_memory = PresidioReversibleAnonymizer()

print(anonymizer_with_memory.anonymize("My name is John Doe. Hi John Doe!"))
print(anonymizer_with_memory.anonymize("My name is John Doe. Hi John Doe!"))

My name is Ashley Cervantes. Hi Ashley Cervantes!
My name is Ashley Cervantes. Hi Ashley Cervantes!


`PresidioReversibleAnonymizer`についての詳細は、次のセクションで学ぶことができます。

> You can learn more about `PresidioReversibleAnonymizer` in the next section.


